# Programming Assignment: Анализ тональности отзывов на фильмы: строим простые модели

В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы.

Мы будем использовать стандартный датасет из nltk, уже возникавший в одном из примеров в предыдущих курсах. Для того, чтобы импортировать необходимый модуль, напишите:

In [1]:
from nltk.corpus import movie_reviews

In [2]:
def write_result(file_name, result):
    file_obj = open(file_name, "w")
    file_obj.write(result)
    file_obj.close()

Чтобы получить id-шники негативных и позитивных отзывов:

In [3]:
negids = movie_reviews.fileids("neg")
posids = movie_reviews.fileids("pos")

Чтобы получить список негативных отзывов:

In [4]:
negfeats = [movie_reviews.raw(fileids=[f]) for f in negids] # words
posfeats = [movie_reviews.raw(fileids=[f]) for f in posids]

## Инструкция по выполнению
В некоторых пунктах нужно получить ответ - число или строку, которые будет нужно набирать в текстовых файлах и прикреплять в ответах на вопросы. *Десятичные дроби записывайте через точку*.

1.Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.

In [5]:
negative_count = len(negids)
positive_count = len(posids)
neg_classes = [0] * negative_count
pos_classes = [1] * positive_count
classes = neg_classes + pos_classes
all_texts = negfeats + posfeats

2.Подсчитайте количество отзывов в выборке.

In [6]:
reviews_count = negative_count + positive_count
write_result("reviews_count.txt", str(reviews_count))
reviews_count

2000

3.Подсчитайте долю класса 1 в выборке.

In [7]:
positive_proportion = float(positive_count)/float(reviews_count)
write_result("positive_proportion.txt", str(positive_proportion))
positive_proportion

0.5

4.Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import random

In [9]:
vectorizer = CountVectorizer().fit(all_texts)
vectorized_texts = vectorizer.transform(all_texts)
write_result("features_count.txt", str(vectorized_texts.shape[1]))
vectorized_texts.shape

(2000, 39659)

5.Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.

In [10]:
logistic_pipe = Pipeline([("vectorize", CountVectorizer()), ("model", LogisticRegression())])
accuracy_scores = cross_val_score(logistic_pipe, all_texts, classes, scoring="accuracy")
write_result("accuracy_score.txt", str(accuracy_scores[0]))
accuracy_scores

array([0.81437126, 0.84684685, 0.84684685])

6.Аналогично accuracy, оцените качество по ROC AUC.

In [11]:
roc_auc_scores = cross_val_score(logistic_pipe, all_texts, classes, scoring="roc_auc")
write_result("roc_auc_score.txt", str(roc_auc_scores[0]))
roc_auc_scores

array([0.9006239 , 0.91283175, 0.91887383])

7.Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.

In [12]:
model = LogisticRegression().fit(vectorized_texts, classes)

In [13]:
import numpy as np
main_features = list(map(
    lambda feature: feature[0],
    sorted(
        zip(vectorizer.get_feature_names(), np.abs(model.coef_[0])),
        key=lambda feature: feature[1],
        reverse=True)[:2]))
main_features_str = " ".join(main_features)
write_result("main_features.txt", main_features_str)
main_features_str

'bad unfortunately'